# Fake news classifier using Bidirectional LSTM

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/Users/shaikmohammadasrarahammad/Downloads/data_science/NLP/data/bilstm.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
df = df.dropna()

In [7]:
df.shape

(18285, 5)

In [8]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [9]:
# getting independent features
X = df.drop('label',axis=1)

# getting dependent features
y = df['label']

In [10]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [11]:
import tensorflow as tf

In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [13]:
# vocabulary size
voc_size = 5000

In [14]:
messages = X.copy()

In [15]:
import nltk
import re
from nltk.corpus import stopwords

In [16]:
messages.reset_index(inplace=True)

In [17]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

for i in range(0,len(messages)):
    # print(i)
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [18]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

# One hot encoding

In [19]:
onehot_rep = [one_hot(words,voc_size)for words in corpus]
print(onehot_rep[0])

[2183, 1662, 4802, 4551, 845, 3081, 2242, 2932, 2881, 1496]


# Creating an embedding layer

In [20]:
sent_len = 20
embedded_docs = pad_sequences(onehot_rep,padding='pre',maxlen=sent_len)

In [21]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2183,
       1662, 4802, 4551,  845, 3081, 2242, 2932, 2881, 1496], dtype=int32)

In [22]:
# Creating a model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_len))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
import numpy as np

In [24]:
X_final = np.array(embedded_docs)
y = np.array(y)

In [25]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y,test_size=0.33,random_state=42)

In [26]:
# model training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10


2023-02-28 19:50:23.085048: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


383/383 [==============================] - 6s 11ms/step - loss: 0.2568 - accuracy: 0.8849 - val_loss: 0.1974 - val_accuracy: 0.9163
Epoch 2/10
383/383 [==============================] - 4s 12ms/step - loss: 0.1328 - accuracy: 0.9495 - val_loss: 0.2103 - val_accuracy: 0.9216
Epoch 3/10
383/383 [==============================] - 5s 14ms/step - loss: 0.0893 - accuracy: 0.9665 - val_loss: 0.2280 - val_accuracy: 0.9115
Epoch 4/10
383/383 [==============================] - 5s 13ms/step - loss: 0.0619 - accuracy: 0.9777 - val_loss: 0.3102 - val_accuracy: 0.9120
Epoch 5/10
383/383 [==============================] - 5s 13ms/step - loss: 0.0376 - accuracy: 0.9874 - val_loss: 0.3799 - val_accuracy: 0.9094
Epoch 6/10
383/383 [==============================] - 5s 12ms/step - loss: 0.0241 - accuracy: 0.9924 - val_loss: 0.3928 - val_accuracy: 0.9148
Epoch 7/10
383/383 [==============================] - 5s 12ms/step - loss: 0.0212 - accuracy: 0.9937 - val_loss: 0.4453 - val_accuracy: 0.9095
Epoch 8/10

In [27]:
y_pred = model.predict(X_test)

189/189 [==============================] - 1s 3ms/step


In [28]:
y_pred = np.where(y_pred>=0.5,1,0)

In [29]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[3069,  350],
       [ 178, 2438]])

In [30]:
accuracy_score(y_test,y_pred)

0.9125103562551782